# CMU 17-400/17-700 auto-graded notebook

Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

---

#### CMU 17400/17700 Data Science and Machine Learning at Scale

## Homework 1: Wikipedia

In [ ]:
# Who did you collaborate with on this assignment? 
# if no one, collaborators should contain an empty string,
# else list your collaborators below

# collaborators = [""]
# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
try:
    collaborators
except:
    raise AssertionError("you did not list your collaborators, if any")   


# ** 1. Wikipedia**

In this exercise, you will familiarize yourself with Spark by exploring full-text Wikipedia articles. The volume of unstructured text in existence is growing dramatically, and Spark is an excellent tool for analyzing this type of data.

Gauging how popular a programming language is important for companies judging whether or not they should adopt an emerging programming language. For that reason, industry analyst firm RedMonk has bi-annually computed a ranking of programming language popularity using a variety of data sources, typically from websites like GitHub and StackOverflow. See their [top-20 ranking for June 2016](http://redmonk.com/sogrady/2016/07/20/language-rankings-6-16/) as an example.


## During this question set we will cover:
* *Part 1:* Creating a base RDD and loading data
* *Part 2:* Counting with aggregations
* *Part 3:* Using an inverted index
* *Part 4:* Directly ranking with `reduceByKey()`

> Note that for reference, you can look up the details of the relevant methods in:
> * [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

## Part 1: Creating a base RDD and loading data

In this part of the lab, we will explore creating a base RDD with `parallelize` and using pair RDDs to count words.

### (1a) Create a base RDD
We'll start by generating a base RDD by using a Python list and the `sc.parallelize` method.  Then we'll print out the type of the base RDD.

In [ ]:
# YOU CAN MOST LIKELY IGNORE THIS CELL. This is only of use for running this notebook locally.

# THIS CELL DOES NOT NEED TO BE RUN ON DATABRICKS. 
# Note that Databricks already creates a SparkContext for you, so this cell can be skipped.
import findspark
findspark.init()
import pyspark
from pyspark.sql import SQLContext
sc = pyspark.SparkContext(appName="hw")
sqlContext = SQLContext(sc)

print("spark context started")

In [ ]:
langsList = ['java', 'python', 'ruby', 'perl', 'scala', 'haskell', 'clojure', 'groovy']
langsRDD = sc.parallelize(langsList, 4)
# Print out the type of wordsRDD
print(type(langsRDD))

### (1b) Capitalize and test

Let's use a `map()` transformation to capitalize each string in the base RDD we just created. We'll define a Python function that properly captalizes the word.  Please replace `<FILL IN>` with your solution. After you have defined `capitalize` you can run the cell which contains a test.  If you implementation is correct it will not print out anything; otherwise it will raise an error.

This is the general form that exercises will take, except that no example solution will be provided.  Exercises will include an explanation of what is expected, followed by code cells where one cell will have one or more `<FILL IN>` sections.  The cell that needs to be modified will have `# TODO: Replace <FILL IN> with appropriate code` on its first line.  Once the `<FILL IN>` sections are updated and the code is run, the test cell can then be run to verify the correctness of your solution.  The last code cell before the next markdown section will contain the tests.

In [ ]:
# One way of completing the function
def capitalize(word):
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>

    # YOUR CODE HERE
    raise NotImplementedError()

print(capitalize('java'))

In [ ]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from nose.tools import assert_equal, assert_true

"""Check that makePlural function makes its input plural by adding an s"""
assert_equal(capitalize('scala'), 'Scala')


### (1c) Apply `capitalize` to the base RDD

Now pass each item in the base RDD into a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `capitalize()` function to each element. And then call the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action to see the transformed RDD.

In [ ]:
# TODO: Uncomment the template below and replace <FILL IN> with appropriate code
# capitalizedRDD = langsRDD.map(<FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

print(capitalizedRDD.collect())

In [ ]:
"""Check that makePlural was applied to base RDD and call to collect returns correct output"""
assert_equal(capitalizedRDD.collect(), ['Java', 'Python', 'Ruby', 'Perl', 'Scala', 'Haskell', 'Clojure', 'Groovy'])

### (1d) Pass a `lambda` function to `map`

Let's create the same RDD using a `lambda` function.

In [ ]:
# TODO: Uncomment the template below and replace <FILL IN> with appropriate code
# capitalizedLambdaRDD = langsRDD.map(lambda <FILL IN>)

# YOUR CODE HERE
raise NotImplementedError()

print(capitalizedLambdaRDD.collect())

In [ ]:
"""Check that lambda function applied to base RDD and call to collect returns correct output"""
assert_equal(capitalizedLambdaRDD.collect(),  ['Java', 'Python', 'Ruby', 'Perl', 'Scala', 'Haskell', 'Clojure', 'Groovy'])

### (1e) Load a text file

To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply a `parse()` function using a `map()` transformation to parse a line of the dataset and turn it into a dictionary object containing the article name and title.  Since the file is large we use `first()`, so that we only print 1 line.

In [ ]:
# Just run this code

url = "https://raw.githubusercontent.com/17-700/data/master/hw1/wikipedia.dat"

from pyspark import SparkFiles
sc.addFile(url)

def parse(line):
    subs = "</title><text>"
    i = line.index(subs)
    return { 'title': line[14:i], 'text': line[i+len(subs):len(line)-16] }

wikipediaRdd = sc.textFile("file://" + SparkFiles.get("wikipedia.dat"), 8).map(parse)
wikipediaRdd.first()

## Part 2: Counting with aggregations

We will use a simple metric for determining the popularity of a programming language: the number of Wikipedia articles that mention the language at least once.

An approach you might first consider (we'll see shortly that there are better ways) is based on using the [aggregate()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.aggregate) transformation. As the name implies, the `aggregate()` transformation aggregates the elements of each partition, and then the results for all the partitions, using the given combine functions and a neutral “zero value.”

### (2a) Computing `occurencesOfLang()`

Start by implementing a helper method `occurrencesOfLang` which computes the number of articles that mention the given language at least once. For the sake of simplicity we check that it least one word (delimited by spaces) of the article text is equal to the given language.

*Hint: You can use the mentionsLanguage function defined at the beginning of the following cell*

In [ ]:
def mentionsLanguage(lang, article):
    return lang in article['text'].split(' ')

# TODO: Uncomment the template below and replace <FILL IN> with appropriate code
# initialValue = <FILL IN>

# YOUR CODE HERE
raise NotImplementedError()

def seqOp(lang, count, article):
    """
    Counts the number of language occurences in a partition and returns the running accumulated result.
    Args:
        lang (str): language that we are looking for
        count (int): running tally of the number of times the language has been mentioned so far in the partition
        article (dict): article to evaluate for occurences of the language
    Returns:
        int: updated tally of the number of mentions of the language in the partition
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()

def combOp(countA, countB):
    """
    Combines the number of language mentions in two partitions into a single result.
    Args:
        countA (int): occurences of the language in partition A
        countB (int): occurences of the language in partition B
    Returns:
        int: combined count across both partitions
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
def occurencesOfLang(lang, rdd):
    """
    Returns the number of occurences of a language in a dataset.
    Args:
        lang (str): language to search for
        rdd (rdd): dataset to search against
    Returns:
        int: number of occurences found in dataset
    """
    return rdd.aggregate(initialValue, lambda x, y: seqOp(lang, x, y), combOp)
    
print(occurencesOfLang('Java', wikipediaRdd))

In [ ]:
"""Check that occurencesOfLang returns the correct number of occurences for Java"""
assert_equal(occurencesOfLang('Java', wikipediaRdd), 360)


### (2b) Computing the ranking with `rankLangs()`

Using occurrencesOfLang, implement a method rankLangs which computes a list of pairs where the second component of the pair is the number of articles that mention the language (the first component of the pair is the name of the language).

An example of what rankLangs might return might look like this, for example:

In [ ]:
[("Scala", 999999), ("JavaScript", 1278), ("LOLCODE", 982), ("Java", 42)]

The list should be sorted in descending order. That is, according to this ranking, the pair with the highest second component (the count) should be the first element of the list.

Pay attention to roughly how long it takes to run this part! (It should take tens of seconds.)

In [ ]:
def rankLangs(langs, rdd):
    """
    Returns the number of occurences of a language in a dataset.
    Args:
        langs (list[str]): languages to search for
        rdd (rdd): dataset to search against
    Returns:
        list[str, int]: language counts sorted in descending order
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
print(rankLangs(['Python', 'Scala', 'JavaScript'], wikipediaRdd))

In [ ]:
langs = ['JavaScript', 'Java', 'PHP', 'Python', 'C#', 'C++', 'Ruby', 'CSS', 'Objective-C', 'Perl', 'Scala', 'Haskell', 'MATLAB', 'Clojure', 'Groovy']

correctRanking = [('JavaScript', 977), ('C#', 538), ('Java', 360), ('C++', 199), ('CSS', 191), ('PHP', 165), ('Python', 165),
                  ('Perl', 86), ('Ruby', 80), ('Objective-C', 32), ('Haskell', 32), ('Scala', 24), ('Clojure', 15),
                  ('MATLAB', 14), ('Groovy', 11)]

"""Check that rankLangs returns the correct number of occurences in descending order"""
assert_equal(rankLangs(langs, wikipediaRdd), correctRanking)


## Part 3: Using an inverted index

An inverted index is an index data structure storing a mapping from content, such as words or numbers, to a set of documents. In particular, the purpose of an inverted index is to allow fast full text searches. In our use-case, an inverted index would be useful for mapping from the names of programming languages to the collection of Wikipedia articles that mention the name at least once.

### (3a) Compute an inverted index

Implement the `makeIndex()` method which returns a pair RDD of type ('&lt;lang&gt;', '&lt;[articles]&gt;'). This RDD contains pairs, such that for each language in the given langs list there is at most one pair. Furthermore, the second component of each pair contains the articles that mention the language at least once.

Hint: You might want to use methods `flatMap()` and `groupByKey()` on RDD for this part.

In [ ]:
def makeIndex(langs, rdd):
    """
    Returns the number of occurences of a language in a dataset.
    Args:
        langs (list[str]): languages to search for
        rdd (rdd): dataset to search against
    Returns:
        list[str, int]: language counts sorted in descending order
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
print(len(list(makeIndex(['Python'], wikipediaRdd).first()[1])))

In [ ]:
"""Check that makeIndex returns correct language mappings"""
assert_equal(len(list(makeIndex(['PHP'], wikipediaRdd).first()[1])), 165)



### (3b) Computing the ranking

Use the `makeIndex()` method implemented in the previous part to implement a faster method for computing the language ranking.

Like in Part 2, `rankLangsUsingIndex()` should compute a list of pairs where the second component of the pair is the number of articles that mention the language (the first component of the pair is the name of the language).

Again, the list should be sorted in descending order. That is, according to this ranking, the pair with the highest second component (the count) should be the first element of the list.

Hint: method the `mapValues()` defined for Pair RDDs on could be useful for this part.

Can you notice a performance improvement over the attempt in Part 2? Why?

In [ ]:
def rankLangsUsingIndex(index):
    """
    Returns the number of occurences of a language using the provided index.
    Args:
        index (rdd): index generated by makeIndex
    Returns:
        list[str, int]: language counts sorted in descending order
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
print(rankLangsUsingIndex(makeIndex(['Python', 'Scala', 'JavaScript'], wikipediaRdd)))

In [ ]:
"""Check that rankLangs returns the correct number of occurences in descending order"""
assert_equal(rankLangsUsingIndex(makeIndex(langs, wikipediaRdd)), correctRanking)


## Part 4: Directly ranking with `reduceByKey()`

In the case where the inverted index from above is only used for computing the ranking and for no other task (full-text search, say), it is more efficient to use the `reduceByKey()` method to compute the ranking directly, without first computing an inverted index. Note that the `reduceByKey()` method is only defined for RDDs containing pairs (each pair is interpreted as a key-value pair).

### (4a) Implementing `rankLangsReduceByKey()`

Implement the `rankLangsReduceByKey()` method, this time computing the ranking without the inverted index, using `reduceByKey()`.

Like in Parts 2 and 3, `rankLangsReduceByKey()` should compute a list of pairs where the second component of the pair is the number of articles that mention the language (the first component of the pair is the name of the language).

Again, the list should be sorted in descending order. That is, according to this ranking, the pair with the highest second component (the count) should be the first element of the list.

Can you notice an improvement in performance compared to measuring both the computation of the index and the computation of the ranking in the previous attempts? If so, can you think of a reason?

In [ ]:
def rankLangsReduceByKey(langs, rdd):
    """
    Returns the number of occurences of a language in a dataset.
    Args:
        langs (list[str]): languages to search for
        rdd (rdd): dataset to search against
    Returns:
        list[str, int]: language counts sorted in descending order
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    # return <FILL IN>
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
print(rankLangsReduceByKey(['Python', 'Scala', 'JavaScript'], wikipediaRdd))

In [ ]:
"""Check that rankLangsReduceByKey returns the correct number of occurences in descending order"""
assert_equal(rankLangsReduceByKey(langs, wikipediaRdd), correctRanking)


## Export the notebook as an IPython notebook, then submit it to Gradescope!